Create an SIC <-> ISIC Crosswalk by hand.  This gets wiped every time the main DERA ingestion runs, but can be run after that any number of times

In [1]:
from dotenv import dotenv_values, load_dotenv
from osc_ingest_trino import *
import os
import pathlib

Load Environment Variables

In [2]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [3]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

In [4]:
import osc_ingest_trino as osc

trino_bucket = osc.attach_s3_bucket("S3_DEV")

Open a Trino connection using JWT for authentication

In [5]:
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sec_dera'

In [6]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

('aicoe_osc_demo',)
('company_data',)
('default',)
('defaultschema1',)
('demo',)
('eje_test_iceberg',)
('epa_frs',)
('epa_ghgrp',)
('epacems',)
('epacems_y95_al',)
('essd',)
('ghgrp_demo',)
('gleif',)
('gleif_mdt',)
('iceberg_demo',)
('information_schema',)
('ingest_schema',)
('iso3166',)
('itr_mdt',)
('metastore',)
('metastore_iceberg',)
('osc_corp_data',)
('physical_risk_project',)
('pudl',)
('rmi_20210929',)
('rmi_20211120',)
('rmi_utility_transition_hub',)
('sec_dera',)
('sfi_geoasset',)
('team1',)
('team2',)
('testaccessschema1',)
('testdb',)
('urgentem',)
('us_census',)
('wri',)
('wri_demo',)
('wri_dev',)
('wri_gppd',)
('wri_gppd_md',)
('wri_new',)
('wri_test',)


Enter the Pandas!

In [7]:
import pandas as pd

For now, create SIC -> ISIC crosswalk by hand

In [8]:
sic_isic = {
    1400: 1410, # Mining
    2911: 1920, # Petroleum refining
    3714: 2910, # Motor Vehicle Manufacturing
    3829: 3190, # Measuring & Controlling Devices (Midwest Energy Emissions Corp (MEEC))
    3310: 2410, # STEEL WORKS, BLAST FURNACES & ROLLING & FINISHING MILLS
    3311: 2410, # ???
    3312: 2410, # STEEL WORKS, BLAST FURNACES & ROLLING MILLS (COKE OVENS)
    3313: 2410, # Iron and Steel Mills and Ferroalloy Mfg.
    3315: 2410, # Steel Wire Drawing
    3316: 2410, # Rolled Steel Shape Mfg.
    3317: 2410, # Iron and Steel Pipe and Tube Mfg. from Purchased Steel (should be relatively larger S3 emissions, lower S1+S2 emissions per ton)
    4911: 4010, # Electricity Generation
    4931: 4010, # Electricity Generation
    4932: 4010, # Electricity Generation
    4991: 4010, # Typo?
}

df = pd.DataFrame.from_dict(sic_isic, orient='index', columns=['isic'])
df.reset_index(inplace=True)
df.rename(columns={'index':'sic'}, inplace=True)
df

,sic,isic
0,1400,1410
1,2911,1920
2,3714,2910
3,3829,3190
4,3310,2410
5,3311,2410
6,3312,2410
7,3313,2410
8,3315,2410
9,3316,2410


In [9]:
osc.drop_unmanaged_table(ingest_catalog, ingest_schema, 'sic_isic', engine, trino_bucket)
osc.ingest_unmanaged_parquet(df, ingest_schema, 'sic_isic', trino_bucket)
tabledef = unmanaged_parquet_tabledef(df, ingest_catalog, ingest_schema, 'sic_isic', trino_bucket)
qres = engine.execute(tabledef)
qres.fetchall()

[(True,)]